# Using LSTMs to generate Data

## Process of generating Data

1. Input data is converted using language model
2. Learn this probalbility distribution.

During testing time 
- After learning sample for the next character.

- To control the sampling we introduce a temperature factor, which will allow us to control amount of sampling that we are doing. 

- Use temperature factor to control amount of randomness.
Higher temperature factor means we get more novel and interesting text but it is inaccurate.

- Lower temperature factor means consistent text but that will not make the text interesting.

In [0]:
import numpy as np

In [0]:
def reweight_distribution(original_distribution, temperature=0.5):
    # Original distribution is a probability distrubtion hence sum of all is 1.
    # Temperature will quantify the distribution
    distribution = np.log(original_distribution / temperature)
    distributoion = np.exp(distribution)
    return (distribution / np.sum(distribution))  
    # Returns reweighted distribution. Sum of distribution 
    # may not be 1, so we need to divide it.


# Implementating Character level LSTM

In [0]:
from keras.utils import get_file
import numpy as np
from keras import layers
from keras.models import Sequential
from keras import optimizers
import random
import sys

In [0]:
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()

In [0]:
# print(text)

In [6]:
print("Corpus Text: ", len(text))

Corpus Text:  600893


Next, you’ll extract partially overlapping sequences of length maxlen, one-hot encode them, and pack them in a 3D Numpy array x of shape. `(sequences, maxlen, unique_characters)`

Simultaneously, you’ll prepare an array y containing the corresponding
targets: the one-hot-encoded characters that come after each extracted
sequence.



In [0]:
maxlen = 60 # Maximum length of 60 characters
step = 3  # Sampling after every 3 characters
sentences = []   # Our X training data
next_chars = []  # Labels 

In [8]:
for i in range(0, len(text) - maxlen, step):
    # Creating the sentece window, of length maxlen
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])

print("Number of seqeuences : ", len(sentences))

Number of seqeuences :  200278


In [9]:
# List of unique characters
# in the corpus
chars = sorted(list(set(text)))
print("Number of unique characters : ", len(chars))

Number of unique characters :  57


In [0]:
# Dictionary that maps
# unique characters to their
# index in the list “chars
char_indices = {}
for char in chars:
    char_indices[char] = chars.index(char)

In [0]:
# print(sentences)

In [0]:
# print(next_chars)

In [0]:
# print(chars)

In [14]:
print("Vectorizing: ")
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
print(x.shape)
print(y.shape)

# Sentences is a list of individual sentence.
# We go thorough every character in the sentence, mark the corresponding
# value of char in sentence as t.
# We store i as it denotes the batch_number.

# We store the batch_number and the corresponding next character in y.

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1

    y[i, char_indices[next_chars[i]]] = 1 

Vectorizing: 
(200278, 60, 57)
(200278, 57)


In [0]:
model = Sequential()
model.add(layers.LSTM(64, activation='tanh', recurrent_dropout=0.2, dropout=0.2, 
                      input_shape = (maxlen, len(chars)), return_sequences=False))
# model.add(layers.LSTM(32, return_sequences=True))
# model.add(layers.LSTM(16, return_sequences=False))
model.add(layers.Dense(len(chars), activation='softmax'))

In [37]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 64)                31232     
_________________________________________________________________
dense_4 (Dense)              (None, 57)                3705      
Total params: 34,937
Trainable params: 34,937
Non-trainable params: 0
_________________________________________________________________


In [0]:
optimizer = optimizers.Adam(lr=0.015, decay=0.001)
model.compile(loss = 'categorical_crossentropy', optimizer=optimizer)

1. Draw from the model a probability distribution for the next character, given the
generated text available so far.
2. Reweight the distribution to a certain temperature.
3. Sample the next character at random according to the reweighted distribution.
4. Add the new character at the end of the available text.

In [0]:
def sample(preds, temperature=1):
    preds = np.asarray(preds).astype('float64')
    # Use the sampling temperature and control the predictions
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    # Pick according to probability the numpy function does it
    # It samples from a muiltinomial distribution
    probabs = np.random.multinomial(1, preds, 1)
    return(np.argmax(probabs))
    

In [42]:
for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)  # Fits the model for 1 iteration of the data

    # We test our model by sampling a random set of strings from the data
    # Then converting them into one-hot vectors.
    # We predict the next character add the new character to the sampled data
    # Then again continue with the predictions.

    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('-------Generation random text with seed --------')
    print(generated_text)

    for temperature in [0.2,0.5,0.8,1.2,1.4]:
        print('--------Temperature----------', temperature)
        # print('Generated text --------- ')
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1
            
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            # Need to remove one prediction which we actually didn't make
            generated_text = generated_text[1:]
            print(next_char, end = '')

        print()





epoch 1
Epoch 1/1
200278/200278 [==============================] - 163s 812us/step - loss: 1.9356
-------Generation random text with seed --------
ublime, also of the loathsome and dreadful, still greater di
--------Temperature---------- 0.2
sting of the scient of the self the sention of the surtation of the such and the sention of the sention of the such of the sention of the spility and the preation of the decist of the sention and the senting of the sention and in the sention of the section of the spirition of the sentess of the sention of the spirition of the senting the sention of the spile of the some and decertion of the some t
--------Temperature---------- 0.5
o be some in in the ened to as which be some the gre of the sure and will in the prosont the deprition of the proprence that the and live what its and every of parizate and the compals of the sense be the deal and decommand lave of all the section the soul suppally to listerself and and not that the senteress and with as 

KeyboardInterrupt: ignored